In [ ]:
# download the weaviate client
%pip install -U weaviate-client

In [ ]:
import weaviate, os
from weaviate.config import AdditionalConfig, Timeout
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
CLUSTER_URL = os.getenv("CLUSTER_URL")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

# Connect to Weaviate
client = weaviate.connect_to_weaviate_cloud(
	cluster_url=CLUSTER_URL,
	auth_credentials=weaviate.auth.AuthApiKey(API_KEY),
	headers={
		"X-OpenAI-Api-Key": OPENAI_API_KEY,
		"X-Cohere-Api-Key": COHERE_API_KEY
	},
	additional_config=AdditionalConfig(
		timeout=Timeout(init=30, query=60, insert=120)
			)
)

ready = client.is_ready()
server_version = client.get_meta()["version"]
client_version = weaviate.__version__

live = client.is_live()
connected = client.is_connected()

print(f"Weaviate client is ready: {ready}")
print(f"Weaviate Client Version: {client_version}")
print(f"Weaviate Server Version: {server_version}")
print(f"live: {live}")
print(f"Connected: {connected}")

In [ ]:
# Fetching objects from a collection
collection = client.collections.get("<COLLECTION_NAME>")
total_objects = 0
for item in collection.iterator(cache_size=200):
    total_objects += 1
    print(f"Object: {total_objects}")
    print(item.uuid, item.properties)
print(f"Total objects fetched: {total_objects}")

In [ ]:
# List the collection names in Weaviate cluster
try:
    collections = client.collections.list_all()
    if collections:
        print("Collections in Weaviate:")
        # Loop through each collection in the instance
        for collection_name in collections.keys():
            print(f"- {collection_name}")
    else:
        print("No collections found.")
except Exception as e:
    print(f"Error retrieving collections: {e}")

In [ ]:
# Aggregation with groupBy
from weaviate.classes.aggregate import GroupByAggregate

# Get the collection
collection = client.collections.get("<COLLECTION_NAME>")

# Perform the aggregation with groupBy
response = collection.aggregate.over_all(
    group_by=GroupByAggregate(prop="<PROP_NAME>"),
    total_count=True
)

# Print the results
for group in response.groups:
    print(f"Value: {group.grouped_by.value}")
    print(f"Path: {group.grouped_by.prop}")
    print(f"Count: {group.total_count}")

In [ ]:
# Aggregates collection and logs HTTP debug information.
import logging
import datetime
from weaviate.classes.aggregate import GroupByAggregate

# Set up logging to capture HTTP requests
import http.client as http_client
http_client.HTTPConnection.debuglevel = 1

# Configure logging
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)
requests_log = logging.getLogger("requests.packages.urllib3")
requests_log.setLevel(logging.DEBUG)
requests_log.propagate = True

# Record timestamp
current_timestamp = datetime.datetime.now(datetime.timezone.utc).isoformat()
print(f"Starting query at: {current_timestamp}")

try:
    # Get the collection
    collection = client.collections.get("<COLLECTION_NAME>")
    
    # Perform the aggregation with groupBy
    response = collection.aggregate.over_all(
        group_by=GroupByAggregate(prop="<PROPERTY_NAME>"),
        total_count=True
    )
    
    # Print the results
    for group in response.groups:
        print(f"Value: {group.grouped_by.value}")
        print(f"Path: {group.grouped_by.prop}")
        print(f"Count: {group.total_count}")
        
except Exception as e:
    print(f"Error occurred: {str(e)}")
    print(f"Error type: {type(e).__name__}")

print("\nInformation:")
print(f"1. Timestamp: {current_timestamp}")
print("2. URL: Check the debug logs above")
print("3. Headers: Check the debug logs above")

In [ ]:
# Retrieve all collections from Weaviate, process them in batches, and display the results in a DataFrame.
import pandas as pd
def aggregate_collections_in_batches(client, batch_size=10):

    all_collections = client.collections.list_all()
    all_collections = list(all_collections)

    if not all_collections:
        print("No collections found.")
        return pd.DataFrame()

    # This will hold rows for our final DataFrame
    result_rows = []

    # Process collections in smaller batches
    total_collections = len(all_collections)
    print(f"Found {total_collections} collections. Processing in batches of {batch_size}.\n")

    for start_idx in range(0, total_collections, batch_size):
        # Slice the list of collections for this batch
        collections_batch = all_collections[start_idx : start_idx + batch_size]
        print(f"Processing batch {start_idx+1} to {start_idx+len(collections_batch)} ...")
        # Process each collection in this batch
        for collection_name in collections_batch:
            try:
                collection = client.collections.get(collection_name)
            except Exception as e:
                print(f"Error retrieving collection '{collection_name}': {e}")
                result_rows.append({
                    "Collection": collection_name,
                    "Collection Objects": None,
                    "Tenant": "ERROR",
                    "Tenant Objects": str(e)
                })
                continue

            # Attempt to determine if multi-tenancy is enabled and get tenants
            try:
                tenants = collection.tenants.get()
                if tenants:
                    tenant_count = len(tenants)
                    # Put a summary row indicating multi-tenancy
                    result_rows.append({
                        "Collection": collection_name,
                        "Collection Objects": None,
                        "Tenant": "MULTI-TENANCY",
                        "Tenant Objects": f"{tenant_count} tenants"
                    })

                    # For each tenant, get the object count
                    for tenant_name, tenant_obj in tenants.items():
                        try:
                            tenant_collection = collection.with_tenant(tenant_name)
                            response = tenant_collection.aggregate.over_all(total_count=True).total_count
                            result_rows.append({
                                "Collection": collection_name,
                                "Collection Objects": None,
                                "Tenant": tenant_name,
                                "Tenant Objects": response
                            })
                        except Exception as e_inner:
                            print(f"Error aggregating tenant '{tenant_name}' in collection '{collection_name}': {e_inner}")
                            result_rows.append({
                                "Collection": collection_name,
                                "Collection Objects": None,
                                "Tenant": tenant_name,
                                "Tenant Objects": f"ERROR: {e_inner}"
                            })
                else:
                    # No tenants or empty => single-tenancy scenario
                    try:
                        response = collection.aggregate.over_all(total_count=True).total_count
                        result_rows.append({
                            "Collection": collection_name,
                            "Collection Objects": response,
                            "Tenant": None,
                            "Tenant Objects": None
                        })
                    except Exception as e_inner:
                        print(f"Error aggregating collection '{collection_name}': {e_inner}")
                        result_rows.append({
                            "Collection": collection_name,
                            "Collection Objects": "ERROR",
                            "Tenant": None,
                            "Tenant Objects": f"ERROR: {e_inner}"
                        })
            except Exception as e:
                error_msg = str(e).lower()
                if "multi-tenancy is not enabled" in error_msg:
                    # Fallback: get the total count for the entire collection
                    try:
                        response = collection.aggregate.over_all(total_count=True).total_count
                        result_rows.append({
                            "Collection": collection_name,
                            "Collection Objects": response,
                            "Tenant": None,
                            "Tenant Objects": None
                        })
                    except Exception as e_inner:
                        print(f"Error aggregating collection '{collection_name}': {e_inner}")
                        result_rows.append({
                            "Collection": collection_name,
                            "Collection Objects": "ERROR",
                            "Tenant": None,
                            "Tenant Objects": f"ERROR: {e_inner}"
                        })
                else:
                    print(f"Unexpected error retrieving tenants for '{collection_name}': {e}")
                    result_rows.append({
                        "Collection": collection_name,
                        "Collection Objects": "ERROR",
                        "Tenant": "ERROR",
                        "Tenant Objects": str(e)
                    })

    # Finished processing all batches; convert to DataFrame
    result_df = pd.DataFrame(result_rows)
    return result_df

aggregate_collections_in_batches(client, batch_size=2)